In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense,
                                     Dropout, Input, BatchNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

# 1. Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"{len(gpus)} GPU(s) ready with memory growth.")
    except RuntimeError as e:
        print(e)

# 2. Configuration
img_size = (128, 128)
data_path = '/content/data/Final DATASET'
num_classes = 75

# 3. Load Data
X, y = [], []
for label in range(num_classes):
    folder_path = os.path.join(data_path, str(label))
    if not os.path.exists(folder_path):
        print(f"Missing folder: {folder_path}")
        continue

    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            X.append(img)
            y.append(label)

X = np.array(X).reshape(-1, img_size[0], img_size[1], 1).astype('float32') / 255.0
y = to_categorical(np.array(y), num_classes)

# 4. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 5. Data Augmentation Function
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    return image, label

# 6. tf.data Datasets
batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
    .shuffle(buffer_size=2048) \
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)) \
    .batch(batch_size) \
    .prefetch(tf.data.AUTOTUNE)

# 7. Build Model
model = Sequential([
    Input(shape=(img_size[0], img_size[1], 1)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
])

# 8. Compile Model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 9. Callbacks
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=4, verbose=1, min_lr=1e-6)

early_stop = EarlyStopping(
    monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1)

# 10. Train Model
model.fit(train_dataset,
          epochs=100,
          validation_data=val_dataset,
          callbacks=[lr_scheduler, early_stop])

# 11. Evaluate
loss, acc = model.evaluate(val_dataset)
print(f"\nTest Accuracy: {acc * 100:.2f}%")

# 12. Additional Metrics
y_true = np.argmax(y_test, axis=1)
y_pred_probs = model.predict(X_test, batch_size=batch_size)
y_pred = np.argmax(y_pred_probs, axis=1)

accuracy = accuracy_score(y_true, y_pred) * 100
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("\nEvaluation Metrics:")
print(f"Accuracy       : {accuracy:.2f}%")
print(f"Precision (avg): {precision:.4f}")
print(f"Recall    (avg): {recall:.4f}")
print(f"F1 Score  (avg): {f1:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(y_true, y_pred))

# 13. Save Model
model.save("/content/drive/MyDrive/Colab Notebooks/vattaeluthu_tamil_advanced.keras")


1 GPU(s) ready with memory growth.
Epoch 1/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 23s 72ms/step - accuracy: 0.0338 - loss: 4.5594 - val_accuracy: 0.0126 - val_loss: 9.8946 - learning_rate: 1.0000e-04
Epoch 2/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.1614 - loss: 3.4827 - val_accuracy: 0.0467 - val_loss: 5.8692 - learning_rate: 1.0000e-04
Epoch 3/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.2864 - loss: 2.8342 - val_accuracy: 0.2700 - val_loss: 2.9360 - learning_rate: 1.0000e-04
Epoch 4/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.3864 - loss: 2.3140 - val_accuracy: 0.5540 - val_loss: 1.7165 - learning_rate: 1.0000e-04
Epoch 5/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.4652 - loss: 1.9488 - val_accuracy: 0.6473 - val_loss: 1.2791 - learning_rate: 1.0000e-04
Epoch 6/100
193/193 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.5261 - loss: 1.6922 - val_accuracy: 0.7141 - val_loss: 1.0978 - learning_rate: 1.0000e-04
Epoch 7/100
1

In [ ]:
!unzip -q "/content/drive/MyDrive/Colab Notebooks/DATASET200.zip" -d /content/data

replace /content/data/Final DATASET/0/image_0001.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!find "/content/data/Final DATASET" -mindepth 1 -maxdepth 1 -type d | wc -l


75


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 434.2 kB/s eta 0:00:00
